In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [57]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [3]:
rbn = db.routes_by_name()
route = rbn["220"].id
entries = db.snapshots(r=route, d=dt.date(2019, 2, 18))
stops_by_name = db.stops_by_name()
timetables = api.timetables("220", stops_by_name)
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [4]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]), reverse=True)]

In [5]:
pvars = sorted(list(
    prediction.possible_variants(
        prediction.drop_duplicate_positions(entries_by_vehicle[vehicles_by_entry_count[1]]), 
        timetable_variants
    )),
    key = lambda t: t[0].poll_time)

In [6]:
order_pvars = list(prediction.check_variant_order(pvars))

In [84]:
precheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(pvars, 200, 300)]
precheck

[(6,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    12,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    18,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    21,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    11,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    17,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    20,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.8

In [85]:
postcheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(order_pvars, 200, 300)]
postcheck

[(3,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:07.608235',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:27.624700',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:47.641593',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:03:07.658397',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East

In [58]:
route_sections = {v: list(prediction.route_sections(v.stops)) for v in timetable_variants}
stop_shaped_entries = [(e, {v: {t[1] for t in ts} for v, ts in util.dict_collect_set(vs, lambda tpl: tpl[0]).items()}) for (e, vs) in order_pvars]
times = prediction.stop_times(stop_shaped_entries, route_sections)
section_times = prediction.section_times(stop_shaped_entries, route_sections)
journeys = prediction.journeys(section_times)

In [49]:
positions = [(s.poll_time, ps) for s, ps in stop_shaped_entries]
positions

[(datetime.datetime(2019, 2, 18, 8, 16, 14, 859009),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(2019, 2, 18, 8, 22, 35, 74702),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(2019, 2, 18, 8, 25, 35, 130512),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(201

In [50]:
js = list(journeys.values())[0]
journey = js[0]
journey

[(0,
  (Nothing(), Just(value=datetime.datetime(2019, 2, 18, 8, 16, 14, 859009))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)))),
 (1,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473)))),
 (2,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 28, 15, 238443)))),
 (3,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(201

In [51]:
padded = prediction.pad_journeys(journeys)
pjs = list(padded.values())[0]
pjourney = pjs[0]
pjourney

[(0,
  (Nothing(), Just(value=datetime.datetime(2019, 2, 18, 8, 16, 14, 859009))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)))),
 (1,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473)))),
 (2,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 28, 15, 238443)))),
 (3,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(201

In [165]:
[len(t) for v, ts in times.items() for t in ts]

[17633, 16064, 17497, 16706, 15302, 12582]

In [48]:
for (variant, these_times) in section_times.items():
    print(f"{variant}:")
    for j, t in enumerate(these_times):
        p, (t1, t2), (t3, t4) = t
        print(f"Section {p} (index {j})")
        print(f"Last before: {t1.map(lambda t: t.time().isoformat())}")
        print(f"First in: {t2.map(lambda t: t.time().isoformat())}")
        print(f"Last in: {t3.map(lambda t: t.time().isoformat())}")
        print(f"First after: {t4.map(lambda t: t.time().isoformat())}")

(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden):
Section 0 (index 0)
Last before: Nothing()
First in: Just(value='08:16:14.859009')
Last in: Just(value='08:25:35.130512')
First after: Just(value='08:26:15.164377')
Section 1 (index 1)
Last before: Just(value='08:25:35.130512')
First in: Just(value='08:26:15.164377')
Last in: Just(value='08:26:15.164377')
First after: Just(value='08:26:35.181473')
Section 2 (index 2)
Last before: Just(value='08:26:15.164377')
First in: Just(value='08:26:35.181473')
Last in: Just(value='08:27:35.216270')
First after: Just(value='08:28:15.238443')
Section 3 (index 3)
Last before: Just(value='08:26:15.164377')
First in: Just(value='08:26:35.181473')
Last in: Just(value='08:27:35.216270')
First after: Just(value='08:28:15.238443')
Section 4 (index 4)
Last before: Just(value='08:27:35.216270')
First in: Just(value='08:28:15.238443')
Last in: Just(value='08:28:15.238443')
First after: Just(value='08:28:35.244090')
Section 5 (index 5)
Last b

Section 31 (index 309)
Last before: Just(value='19:15:38.644633')
First in: Just(value='19:16:18.670080')
Last in: Just(value='19:16:18.670080')
First after: Just(value='19:16:58.702842')
Section 32 (index 310)
Last before: Just(value='19:16:18.670080')
First in: Just(value='19:16:58.702842')
Last in: Just(value='19:17:18.719028')
First after: Just(value='19:17:38.735453')
Section 33 (index 311)
Last before: Just(value='19:17:18.719028')
First in: Just(value='19:17:38.735453')
Last in: Just(value='19:17:38.735453')
First after: Just(value='19:17:58.751950')
Section 34 (index 312)
Last before: Just(value='19:17:38.735453')
First in: Just(value='19:17:58.751950')
Last in: Just(value='19:18:18.768441')
First after: Just(value='19:18:58.790546')
Section 36 (index 313)
Last before: Just(value='19:18:18.768441')
First in: Just(value='19:18:58.790546')
Last in: Just(value='19:18:58.790546')
First after: Just(value='19:19:18.806942')
Section 37 (index 314)
Last before: Just(value='19:18:58.790

First after: Just(value='09:46:38.539195')
Section 112 (index 81)
Last before: Just(value='09:45:58.506427')
First in: Just(value='09:46:38.539195')
Last in: Just(value='09:46:38.539195')
First after: Just(value='09:46:58.555944')
Section 114 (index 82)
Last before: Just(value='09:46:38.539195')
First in: Just(value='09:46:58.555944')
Last in: Just(value='09:46:58.555944')
First after: Just(value='09:47:18.572388')
Section 116 (index 83)
Last before: Just(value='09:46:58.555944')
First in: Just(value='09:47:58.606191')
Last in: Just(value='09:47:58.606191')
First after: Just(value='09:48:18.622745')
Section 118 (index 84)
Last before: Just(value='09:47:58.606191')
First in: Just(value='09:48:18.622745')
Last in: Just(value='09:48:18.622745')
First after: Just(value='09:48:38.639598')
Section 120 (index 85)
Last before: Just(value='09:48:18.622745')
First in: Just(value='09:48:38.639598')
Last in: Just(value='09:53:38.861445')
First after: Just(value='09:53:58.878389')
Section 121 (inde

Last in: Just(value='23:06:25.408584')
First after: Just(value='23:07:45.454963')
Section 0 (index 465)
Last before: Just(value='00:15:42.974391')
First in: Just(value='00:16:02.992802')
Last in: Just(value='00:16:02.992802')
First after: Just(value='00:29:43.597734')
Section 6 (index 466)
Last before: Just(value='00:16:02.992802')
First in: Just(value='00:30:23.634925')
Last in: Just(value='00:30:23.634925')
First after: Just(value='00:30:43.638638')
Section 7 (index 467)
Last before: Just(value='00:30:23.634925')
First in: Just(value='00:30:43.638638')
Last in: Just(value='00:30:43.638638')
First after: Just(value='00:31:23.670644')
Section 9 (index 468)
Last before: Just(value='00:30:43.638638')
First in: Just(value='00:31:23.670644')
Last in: Just(value='00:31:23.670644')
First after: Just(value='00:31:43.686420')
Section 11 (index 469)
Last before: Just(value='00:31:23.670644')
First in: Just(value='00:31:43.686420')
Last in: Just(value='00:31:43.686420')
First after: Just(value='

First in: Just(value='16:54:33.749468')
Last in: Just(value='16:55:13.778458')
First after: Just(value='16:55:33.790527')
Section 96 (index 248)
Last before: Just(value='16:55:13.778458')
First in: Just(value='16:57:33.869727')
Last in: Just(value='16:57:33.869727')
First after: Just(value='16:58:13.875342')
Section 97 (index 249)
Last before: Just(value='16:57:33.869727')
First in: Just(value='16:59:13.884286')
Last in: Just(value='16:59:13.884286')
First after: Just(value='16:59:33.899953')
Section 97 (index 250)
Last before: Just(value='16:59:13.884286')
First in: Just(value='17:00:53.949165')
Last in: Just(value='17:00:53.949165')
First after: Just(value='17:01:13.964350')
Section 98 (index 251)
Last before: Just(value='17:00:53.949165')
First in: Just(value='17:01:13.964350')
Last in: Just(value='17:01:53.994414')
First after: Just(value='17:01:53.994414')
Section 99 (index 252)
Last before: Just(value='17:01:53.994414')
First in: Just(value='17:01:53.994414')
Last in: Just(value=

Last before: Just(value='11:28:42.405654')
First in: Just(value='11:29:02.420050')
Last in: Just(value='11:29:22.436567')
First after: Just(value='11:29:42.452738')
Section 90 (index 63)
Last before: Just(value='11:29:22.436567')
First in: Just(value='11:29:42.452738')
Last in: Just(value='11:30:42.500780')
First after: Just(value='11:31:02.514510')
Section 91 (index 64)
Last before: Just(value='11:30:42.500780')
First in: Just(value='11:31:02.514510')
Last in: Just(value='11:31:42.542418')
First after: Just(value='11:32:02.558785')
Section 92 (index 65)
Last before: Just(value='11:31:42.542418')
First in: Just(value='11:32:02.558785')
Last in: Just(value='11:33:22.623450')
First after: Just(value='11:33:42.639274')
Section 93 (index 66)
Last before: Just(value='11:33:22.623450')
First in: Just(value='11:33:42.639274')
Last in: Just(value='11:34:02.655876')
First after: Just(value='11:34:22.669249')
Section 94 (index 67)
Last before: Just(value='11:34:02.655876')
First in: Just(value='

Section 111 (index 341)
Last before: Just(value='22:01:05.222521')
First in: Just(value='22:01:45.256636')
Last in: Just(value='22:01:45.256636')
First after: Just(value='22:02:05.258464')
Section 113 (index 342)
Last before: Just(value='22:01:45.256636')
First in: Just(value='22:02:05.258464')
Last in: Just(value='22:02:25.275692')
First after: Just(value='22:02:45.292894')
Section 115 (index 343)
Last before: Just(value='22:02:25.275692')
First in: Just(value='22:02:45.292894')
Last in: Just(value='22:03:05.300820')
First after: Just(value='22:03:25.306535')
Section 116 (index 344)
Last before: Just(value='22:03:05.300820')
First in: Just(value='22:03:25.306535')
Last in: Just(value='22:03:25.306535')
First after: Just(value='22:03:45.323801')
Section 117 (index 345)
Last before: Just(value='22:03:25.306535')
First in: Just(value='22:03:45.323801')
Last in: Just(value='22:04:05.340729')
First after: Just(value='22:04:45.368076')
Section 118 (index 346)
Last before: Just(value='22:04:

Last in: Just(value='15:25:30.630416')
First after: Just(value='15:26:30.661465')
Section 112 (index 168)
Last before: Just(value='15:25:30.630416')
First in: Just(value='15:27:30.670574')
Last in: Just(value='15:27:30.670574')
First after: Just(value='15:27:50.674072')
Section 113 (index 169)
Last before: Just(value='15:27:30.670574')
First in: Just(value='15:27:50.674072')
Last in: Just(value='15:29:10.686089')
First after: Just(value='15:29:50.691214')
Section 114 (index 170)
Last before: Just(value='15:29:10.686089')
First in: Just(value='15:29:50.691214')
Last in: Just(value='15:29:50.691214')
First after: Just(value='15:30:10.694604')
Section 115 (index 171)
Last before: Just(value='15:29:50.691214')
First in: Just(value='15:30:10.694604')
Last in: Just(value='15:30:50.697645')
First after: Just(value='15:32:50.739127')
Section 10 (index 172)
Last before: Just(value='17:15:54.469086')
First in: Just(value='17:16:34.485815')
Last in: Just(value='17:17:54.507674')
First after: Just

First in: Just(value='10:39:00.542404')
Last in: Just(value='10:40:00.569389')
First after: Just(value='10:40:20.585484')
Section 9 (index 4)
Last before: Just(value='10:40:00.569389')
First in: Just(value='10:40:20.585484')
Last in: Just(value='10:40:20.585484')
First after: Just(value='10:40:40.601678')
Section 10 (index 5)
Last before: Just(value='10:40:20.585484')
First in: Just(value='10:40:40.601678')
Last in: Just(value='10:40:40.601678')
First after: Just(value='10:41:20.633816')
Section 14 (index 6)
Last before: Just(value='10:40:40.601678')
First in: Just(value='10:46:00.826737')
Last in: Just(value='10:46:00.826737')
First after: Just(value='10:46:40.859196')
Section 15 (index 7)
Last before: Just(value='10:46:00.826737')
First in: Just(value='10:46:40.859196')
Last in: Just(value='10:46:40.859196')
First after: Just(value='10:47:00.864845')
Section 16 (index 8)
Last before: Just(value='10:46:40.859196')
First in: Just(value='10:47:00.864845')
Last in: Just(value='10:48:00.8

Last in: Just(value='18:37:57.296742')
First after: Just(value='18:38:37.322458')
Section 87 (index 226)
Last before: Just(value='18:37:57.296742')
First in: Just(value='18:38:37.322458')
Last in: Just(value='18:38:37.322458')
First after: Just(value='18:38:57.338503')
Section 88 (index 227)
Last before: Just(value='18:38:37.322458')
First in: Just(value='18:38:57.338503')
Last in: Just(value='18:39:37.369769')
First after: Just(value='18:39:57.385852')
Section 90 (index 228)
Last before: Just(value='18:39:37.369769')
First in: Just(value='18:39:57.385852')
Last in: Just(value='18:39:57.385852')
First after: Just(value='18:40:17.401821')
Section 91 (index 229)
Last before: Just(value='18:39:57.385852')
First in: Just(value='18:40:17.401821')
Last in: Just(value='18:40:17.401821')
First after: Just(value='18:40:37.410431')
Section 92 (index 230)
Last before: Just(value='18:40:17.401821')
First in: Just(value='18:40:37.410431')
Last in: Just(value='18:42:57.475020')
First after: Just(val

In [74]:
first_snapshot = stop_shaped_entries[40]
positions = util.first(first_snapshot[1].values()).value
positions
[(v.stops[0].name, ts) for s in stop_shaped_entries for v, ts in s[1].items()]

[('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange

In [61]:
to_time = lambda d: d.time().isoformat()
for variant, these_stop_times in times.items():
    print(f"Variant: {variant}")
    print(f"{len(these_stop_times)} journeys")
    for trip_number, trip_times in enumerate(these_stop_times):
        for stop_number, stop_time in enumerate(trip_times):
            to_time = lambda d: d.time().isoformat()
            print(f"- {trip_number}, {stop_number:2}, {variant.stops[stop_number].name:40}"
                  f" {stop_time.last_before.map(to_time).or_else(''):15}, {stop_time.first_after.map(to_time).or_else(''):15}")

Variant: (route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden)
8 journeys
- 0,  0, Ovens (Grange Road Terminus)                            , 08:26:15.164377
- 0,  1, Ovens (Opp Grange Manor)                 08:26:15.164377, 08:28:15.238443
- 0,  2, Killumney Road (Kilumney Cross)          08:26:15.164377, 08:28:15.238443
- 0,  3, Ovens (EMC Terminus)                     08:28:15.238443, 08:30:15.317625
- 0,  4, Ballincollig West (Classes Lake)         08:31:55.403588, 08:33:35.488142
- 0,  5, Ballincollig West (Opp Aylsbury Estate)  08:33:35.488142, 08:34:55.547810
- 0,  6, Ballincollig West (Opp Old Quarry)       08:34:15.522705, 08:36:15.610242
- 0,  7, Ballincollig West (Opp Coolroe Heights)  08:35:55.592807, 08:36:15.610242
- 0,  8, Ballincollig West (Op White Horse Bar)   08:37:35.671351, 08:38:35.709878
- 0,  9, Ballincollig (Opp Jctn Barrys Road)      08:38:15.692639, 08:39:15.744204
- 0, 10, Ballincollig (Shopping Centre)           08:39:15.744204, 08:39:35.760705

- 0, 62, Ovens (Grange Manor)                     11:49:23.206798, 11:50:43.263617
- 0, 63, Ovens (Grange Road Terminus)             11:51:03.270913, 11:53:43.394486
- 1,  0, Fort Camden                                             ,                
- 1,  1, Camden Road (Southbound)                                ,                
- 1,  2, Camden Road (Point Lane Jctn)                           ,                
- 1,  3, Crosshaven (Village Ctr Northbound)                     ,                
- 1,  4, Crosshaven (Buckleys Bar)                               ,                
- 1,  5, Crosshaven (Opp Boatyard)                               ,                
- 1,  6, Crosshaven (The Grand Apts)                             ,                
- 1,  7, Crosshaven (Hassets Restuarant)                         ,                
- 1,  8, Kilnagleary Rd (O Learys Cross Northboun                ,                
- 1,  9, Carrigaline (Fernlea Ferney Road)                       ,                
- 1,

In [117]:
calculated_times = set()

for v, ts in islice(times.items(), 0, 10):
    for p, stop_times in islice(ts.items(), 0, 100):
        for t in stop_times:
            calculated_times.add(tuple(chain([p, v.stops[p].name], (x for x in t))))
            
            
for (position, name, t1, t2) in sorted(calculated_times, key=lambda tpl: tpl[2]):
    print(f"{position:3}: {name:50} ({t1.time().isoformat()} -> {t2.time().isoformat()})")

AttributeError: 'list' object has no attribute 'items'

In [40]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [43]:
timetables208 = list(api.timetables("208", stops_by_name))
timetables205 = list(api.timetables("205", stops_by_name))

In [46]:
notebook.show_timetables(themap, timetables208)

In [49]:
notebook.show_timetables(themap, timetables205)

In [30]:
notebook.plot_entries(
    themap, 
    (t[0] for t in islice(stop_shaped_entries, 400, 600)), 
    snapshot_to_layer = lambda e: lf.Marker(
        location=(e.latitude, e.longitude),
        draggable=False,
        title=e.poll_time.time().isoformat()
    ),
    clear=False
)

In [7]:
vehicles_by_entry_count

[VehicleId(raw='7338674957838188756'),
 VehicleId(raw='7338674957838188752'),
 VehicleId(raw='7338674957838188925'),
 VehicleId(raw='7338674957838188758'),
 VehicleId(raw='7338674957838188929'),
 VehicleId(raw='7338674957838188928'),
 VehicleId(raw='7338674957838189413'),
 VehicleId(raw='7338674957838188953'),
 VehicleId(raw='7338674957838188804'),
 VehicleId(raw='7338674957838188930'),
 VehicleId(raw='7338674957838189374'),
 VehicleId(raw='7338674957838188809'),
 VehicleId(raw='7338674957838188917'),
 VehicleId(raw='7338674957838188920'),
 VehicleId(raw='7338674957838188879'),
 VehicleId(raw='7338674957838189409'),
 VehicleId(raw='7338674957838189406'),
 VehicleId(raw='7338674957838188838'),
 VehicleId(raw='7338674957838189385'),
 VehicleId(raw='7338674957838189370'),
 VehicleId(raw='7338674957838188805'),
 VehicleId(raw='7338674957838188762'),
 VehicleId(raw='7338674957838188973'),
 VehicleId(raw='7338674957838188927'),
 VehicleId(raw='7338674957838188918'),
 VehicleId(raw='733867495